In [1]:
import scanpy
import pandas as pd
import anndata
import os
import hdf5plugin
import os
import pickle as pkl

import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
import pybiomart
import mygene
from sklearn.preprocessing import MinMaxScaler
import numpy as np



In [3]:
path = "/data_hdd4/hb/lof/data/raw_data_after_preprocessing/norman_for_deg_processing_gsym.h5ad"
adata = anndata.read_h5ad(path)

/home/hb/anaconda3/envs/lof/lib/python3.11/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
metadata = adata.obs
counts_df = adata.to_df()

In [9]:
adata.obs

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,condition
cell_barcode,,,,,,,,
AAACCTGAGAAACCAT-2,CEBPB_NegCtrl0__CEBPB_NegCtrl0,867,55,15.763636,2,True,1,CEBPB
AAACCTGAGAAACCGC-6,FEV_CBFA2T3__FEV_CBFA2T3,746,33,22.606061,6,True,1,FEV_CBFA2T3
AAACCTGAGAAAGTGG-8,TGFBR2_IGDCC3__TGFBR2_IGDCC3_2,1183,53,22.320755,8,True,1,TGFBR2_IGDCC3
AAACCTGAGAAGAAGC-1,NegCtrl0_NegCtrl0__NegCtrl0_NegCtrl0,1252,67,18.686567,1,True,2,control
AAACCTGAGAAGGTTT-7,SNAI1_NegCtrl0__SNAI1_NegCtrl0,1794,94,19.085106,7,True,1,SNAI1
...,...,...,...,...,...,...,...,...
TTTGTCATCTTATCTG-5,NegCtrl0_MAP2K6__NegCtrl0_MAP2K6,1627,59,27.576271,5,True,1,MAP2K6
TTTGTCATCTTGCATT-7,SET_NegCtrl0__SET_NegCtrl0,606,37,16.378378,7,True,1,SET
TTTGTCATCTTGTACT-5,NegCtrl0_COL2A1__NegCtrl0_COL2A1,1929,65,29.676923,5,True,2,COL2A1


In [13]:
adata.var[adata.var.duplicated(keep=False)]

,gene_symbol
gene_name,
MATR3,MATR3
CYB561D2,CYB561D2
RGS5,RGS5
PLSCR3,TMEM256-PLSCR3
LOC127814295,RGS5
TMEM256-PLSCR3,TMEM256-PLSCR3
LOC101928965,CYB561D2
MATR3,MATR3


In [8]:
X = adata.to_df()
X

gene_name,TSPAN6,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,LOC105373553,TEX13D,ADORA3,BISPR,TMEM275,LOC122526780,PRNCR1,LINC02635,LINC02456,H2BC6-AS1
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCAT-2,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAACCGC-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAAGTGG-8,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAGAAGC-1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAGGTTT-7,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCTTATCTG-5,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCTTGCATT-7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCTTGTACT-5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
X = X.loc[~X.index.duplicated(keep='first')]

In [23]:
X_T = X.T
X_T = X_T.loc[~X_T.index.duplicated(keep='first')]
X = X_T.T

In [22]:
var = var.loc[~var.index.duplicated(keep='first')]

In [31]:
import hdf5plugin
adata = anndata.AnnData(X=X, obs=adata.obs, var=var)
adata.write_h5ad(
    "/data_hdd4/hb/lof/data/raw_data_after_preprocessing/norman_for_deg_processing_gsym_dup_remove.h5ad",
    compression=hdf5plugin.FILTERS["zstd"]
)

In [34]:
adata.to_df()

gene_name,TSPAN6,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,...,LOC105373553,TEX13D,ADORA3,BISPR,TMEM275,LOC122526780,PRNCR1,LINC02635,LINC02456,H2BC6-AS1
cell_barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAAACCAT-2,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAACCGC-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAAGTGG-8,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAGAAGC-1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACCTGAGAAGGTTT-7,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCTTATCTG-5,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCTTGCATT-7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTCATCTTGTACT-5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
condition = adata.obs.condition[0]

In [38]:
metadata

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,condition
cell_barcode,,,,,,,,
AAACCTGAGAAACCAT-2,CEBPB_NegCtrl0__CEBPB_NegCtrl0,867,55,15.763636,2,True,1,CEBPB
AAACCTGAGAAACCGC-6,FEV_CBFA2T3__FEV_CBFA2T3,746,33,22.606061,6,True,1,FEV_CBFA2T3
AAACCTGAGAAAGTGG-8,TGFBR2_IGDCC3__TGFBR2_IGDCC3_2,1183,53,22.320755,8,True,1,TGFBR2_IGDCC3
AAACCTGAGAAGAAGC-1,NegCtrl0_NegCtrl0__NegCtrl0_NegCtrl0,1252,67,18.686567,1,True,2,control
AAACCTGAGAAGGTTT-7,SNAI1_NegCtrl0__SNAI1_NegCtrl0,1794,94,19.085106,7,True,1,SNAI1
...,...,...,...,...,...,...,...,...
TTTGTCATCTTATCTG-5,NegCtrl0_MAP2K6__NegCtrl0_MAP2K6,1627,59,27.576271,5,True,1,MAP2K6
TTTGTCATCTTGCATT-7,SET_NegCtrl0__SET_NegCtrl0,606,37,16.378378,7,True,1,SET
TTTGTCATCTTGTACT-5,NegCtrl0_COL2A1__NegCtrl0_COL2A1,1929,65,29.676923,5,True,2,COL2A1


In [39]:
metadata.loc[metadata.condition == condition]

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,condition
cell_barcode,,,,,,,,
AAACCTGAGAAACCAT-2,CEBPB_NegCtrl0__CEBPB_NegCtrl0,867,55,15.763636,2,True,1,CEBPB
AAACCTGGTCATGCAT-8,NegCtrl0_CEBPB__NegCtrl0_CEBPB,85,5,17.000000,8,True,1,CEBPB
AAAGATGTCCCTGACT-2,CEBPB_NegCtrl0__CEBPB_NegCtrl0,658,48,13.708333,2,True,1,CEBPB
AAAGATGTCTTGTACT-4,NegCtrl0_CEBPB__NegCtrl0_CEBPB,1383,86,16.081395,4,True,1,CEBPB
AAAGCAATCATTTGGG-8,CEBPB_NegCtrl0__CEBPB_NegCtrl0,1296,74,17.513514,8,True,1,CEBPB
...,...,...,...,...,...,...,...,...
TTTATGCAGTGAACAT-2,NegCtrl0_CEBPB__NegCtrl0_CEBPB,1019,66,15.439394,2,True,1,CEBPB
TTTATGCTCACGACTA-1,NegCtrl0_CEBPB__NegCtrl0_CEBPB,333,15,22.200000,1,True,1,CEBPB
TTTCCTCCATCAGTAC-2,CEBPB_NegCtrl0__CEBPB_NegCtrl0,506,38,13.315789,2,True,1,CEBPB


In [108]:
import numpy as np

save_dir = f'/data_hdd4/hb/lof/data/deg_augmentation/{condition}'
os.makedirs(save_dir, exist_ok=True)

metadata_perturb_total = metadata.loc[metadata.condition == condition].index.tolist()
metadata_ctrl_total = metadata.loc[metadata.condition == 'control'].index.tolist()

perturb_sampling_index_sets = []
ctrl_sampling_index_sets = []
random_int = np.random.randint(1, 100000, size=len(metadata_perturb_total))
for seed_value in random_int:
    np.random.seed(seed_value)

    perturb_sampling_index_sets.append(np.random.choice(metadata_perturb_total, size=10, replace=False))
    ctrl_sampling_index_sets.append(np.random.choice(metadata_ctrl_total, size=10, replace=False))

for perturb_sampling_index, ctrl_sampling_index in zip(perturb_sampling_index_sets, ctrl_sampling_index_sets):
    metadata_ctrl = metadata.loc[ctrl_sampling_index]
    metadata_pertb = metadata.loc[perturb_sampling_index]

    metadata_total = pd.concat([metadata_pertb, metadata_ctrl]).sort_index()
    counts_df_total = counts_df[counts_df.index.isin(metadata_total.index)].sort_index()

    inference = DefaultInference(n_cpus=72)
    dds = DeseqDataSet(
        counts=counts_df_total,
        metadata=metadata_total,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
        ref_level = ['condition', 'control']
    )
    dds.deseq2()
    lfc = dds.varm['LFC']

    column = [x for x in lfc.columns if 'condition' in x]
    lfc = lfc.dropna(axis=0)
    lfc.sort_values(by=column, ascending=False, inplace=True, ignore_index=False)
    lfc.to_csv(f"{save_dir}/{condition}_{seed_value}.csv")
    # lfc['gene_name'] = [gene_name_dict[x] if x in gene_name_dict.keys() else float('nan') for x in lfc.index]
    # lfc = lfc[lfc['gene_name'].notna()]
    # column = lfc.columns[1]
    # lfc = lfc.dropna(axis=0)
    # lfc['gene_id'] = lfc.index
    # lfc.set_index('gene_name', append=False, inplace=True)
    # lfc.sort_values(by=column, ascending=False, inplace=True, ignore_index=False)
    # lfc.to_csv(f"/home/hb/python/lof/deg/norman/{condition}.csv")
    # print(f'{condition}.csv saved!')
    break

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 1.67 seconds.

Fitting dispersion trend curve...
... done in 0.28 seconds.

Fitting MAP dispersions...
... done in 1.50 seconds.

Fitting LFCs...
... done in 1.76 seconds.

Refitting 1 outliers.

/tmp/ipykernel_132827/3071108026.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lfc.sort_values(by=column, ascending=False, inplace=True, ignore_index=False)


In [61]:
print(counts_df_total.index.duplicated().any())
print(metadata_total.index.duplicated().any())


False
False


In [57]:
metadata_total

,guide_identity,read_count,UMI_count,coverage,gemgroup,good_coverage,number_of_cells,condition
cell_barcode,,,,,,,,
ACTTTCAAGATATGCA-2,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,149,9,16.555556,2,True,1,control
CATCAAGGTACCCAAT-8,CEBPB_NegCtrl0__CEBPB_NegCtrl0,1254,56,22.392857,8,True,1,CEBPB
CCACCTACATCGTCGG-4,NegCtrl1_NegCtrl0__NegCtrl1_NegCtrl0,1364,73,18.684932,4,True,1,control
CGATGGCTCACGCGGT-6,NegCtrl10_NegCtrl0__NegCtrl10_NegCtrl0,2871,120,23.925000,6,True,1,control
CGTTGGGAGGCGCTCT-1,NegCtrl11_NegCtrl0__NegCtrl11_NegCtrl0,429,26,16.500000,1,True,1,control
CTAGCCTAGGGTTCCC-1,NegCtrl11_NegCtrl0__NegCtrl11_NegCtrl0,1343,74,18.148649,1,True,1,control
CTGCGGAGTCAACTGT-8,NegCtrl0_NegCtrl0__NegCtrl0_NegCtrl0,2006,91,22.043956,8,True,1,control
CTGCTGTCACCGTTGG-8,CEBPB_NegCtrl0__CEBPB_NegCtrl0,150,11,13.636364,8,True,1,CEBPB
CTTACCGAGGTGTGGT-4,CEBPB_NegCtrl0__CEBPB_NegCtrl0,669,29,23.068966,4,True,1,CEBPB


In [ ]:
already_calculate = []

for condition in [x for x in metadata['condition'].unique() if x not in already_calculate]:
    print(f'{condition} DEG analysis start!')
    metadata_toy = metadata[metadata['condition'].isin([condition, 'control'])]

    """sampling하고 싶을 때 코드
    metadata_toy_ctrl = metadata_toy[metadata_toy['condition']=='control'].sample(n=10)
    metadata_toy_perturb = metadata_toy[metadata_toy['condition']==condition].sample(n=10)
    metadata_toy = pd.concat([metadata_toy_perturb, metadata_toy_ctrl]).sort_index()
    counts_df_toy = counts_df[counts_df.index.isin(metadata_toy.index)].sort_index()
    """
    
    counts_df_toy = counts_df[counts_df.index.isin(metadata_toy.index)]
    inference = DefaultInference(n_cpus=72)
    dds = DeseqDataSet(
        counts=counts_df_toy,
        metadata=metadata_toy,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
        ref_level = ['condition', 'control']
    )
    dds.deseq2()
    lfc = dds.varm['LFC']
    lfc['gene_name'] = [gene_name_dict[x] if x in gene_name_dict.keys() else float('nan') for x in lfc.index]
    lfc = lfc[lfc['gene_name'].notna()]
    column = lfc.columns[1]
    lfc = lfc.dropna(axis=0)
    lfc['gene_id'] = lfc.index
    lfc.set_index('gene_name', append=False, inplace=True)
    lfc.sort_values(by=column, ascending=False, inplace=True, ignore_index=False)
    lfc.to_csv(f"/home/hb/python/lof/deg/norman/{condition}.csv")
    print(f'{condition}.csv saved!')